In [19]:
import tensorflow as tf
import torch
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

!pip install transformers
!pip install bert-tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 124.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 2.7 MB/s eta 0:00:00


In [20]:
from google.colab import drive
drive.mount('/content/gdrive') 
model_save_name = 'haspeede_test_ufficiale.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 

Mounted at /content/gdrive


#Verbalization and davinci

In [21]:
train = pd.read_excel('haspeedee_train.xlsx') 
dev = pd.read_excel('haspeedee_test.xlsx')
conan = pd.read_excel('CONAN Italian.xlsx')
data = pd.concat([train,dev])
data = data.rename(columns={'anonymized_tweet_id':'s'})
data = data.drop(columns=['data_type', 'dataset'])
kg_verbalized = pd.read_excel('kg4haspeede_verbalised.xlsx')

In [ ]:
data

,s,anonymized_text,label
0,217874450618134,@unknown Con tutte le denunce che si sta becca...,1
1,360042217507605,#PRESCRIZIONE : I (t)re magi hanno trovato l’a...,0
2,817917484817935,Il #m5s ha votato contro le #unionicivili addu...,1
3,145836038456701,Che cosa cambia questa #legge? Caro @667517147...,0
4,599642342348521,@800991101440753 Spero rientrino 2000 insegnan...,0
...,...,...,...
555,324165730844406,".@738958761853559 , ma vafangul’\n\nmessaggio ...",1
556,871453511657474,#ENRICOLETTA É DIARREA BERLUSCONICCHIA CHE POR...,1
557,67223505911646,"Ultima Ora: Webuild, nuovi ordini per 2,6 mili...",0
558,517002584005250,#14maggio #PapaFrancesco Occorrono politiche f...,0


In [22]:
conan = conan.rename(columns={'cn_id':'s', 'hateSpeech':'text', 'hsType':'label'})
conan['label'] = 1
conan

,s,text,label
0,ITT1ST0001HS0016CN000010,"Li accogliamo a casa nostra, li manteniamo e l...",1
1,ITT1ST0001HS0016CN000010P1,Noi accogliamo i musulmani e provvediamo a lor...,1
2,ITT1ST0001HS0016CN000010P2,"Noi li ospitiamo nel nostro paese, forniamo un...",1
3,ITT1ST0001HS0016CN000011,"Li accogliamo a casa nostra, li manteniamo e l...",1
4,ITT1ST0001HS0016CN000011P1,Noi accogliamo i musulmani e provvediamo a lor...,1
...,...,...,...
3160,ITT1ST0023HS0055CN001068P1,Una donna che permette di farsi coprire con un...,1
3161,ITT1ST0023HS0055CN001068P2,In Italia non possiamo accogliere donne che de...,1
3162,ITT1ST0023HS0055CN001069,Una donna che accetta di vivere nascosta sotto...,1
3163,ITT1ST0023HS0055CN001069P1,Una donna che permette di farsi coprire con un...,1


In [23]:
#just verbalization
#let's merge the two dataset on the column 'id'
merged_df = pd.merge(data, kg_verbalized[['s', 'triples_verbalised']], on='s') 
#merged_df
#join text and triples
merged_df['text'] = merged_df['anonymized_text'].str.cat(merged_df['triples_verbalised'], sep=' ')
merged_df
#concatenate with the other 'data'
data = data.rename(columns={'anonymized_text':'text'})
merged_df = merged_df.drop(columns=['anonymized_text', 'triples_verbalised'])
# merge the two datasets based on the 'id' column
merged = pd.concat([merged_df, data])
unique = merged.drop_duplicates(subset='s', keep='first')
data = unique

In [24]:
#add conan
data = pd.concat([data,conan])
data

,s,label,text
0,217874450618134,1,@unknown Con tutte le denunce che si sta becca...
1,740684329660693,0,In pochi giorni il #ministro #salvini ha butta...
2,170378636015491,1,Ma se la #carfagna e #crosetto sono più a #sin...
3,906842219275344,1,Nemmeno la tradizionale conferenza stampa di f...
5,48581434415291,1,#decretosicurezza #LeolucaOrlando colui che er...
...,...,...,...
3160,ITT1ST0023HS0055CN001068P1,1,Una donna che permette di farsi coprire con un...
3161,ITT1ST0023HS0055CN001068P2,1,In Italia non possiamo accogliere donne che de...
3162,ITT1ST0023HS0055CN001069,1,Una donna che accetta di vivere nascosta sotto...
3163,ITT1ST0023HS0055CN001069P1,1,Una donna che permette di farsi coprire con un...


In [ ]:
#to add davinci
davinci = pd.read_excel('haspeede_verbalis_davinci.xlsx')
#davinci = davinci.drop(columns=['o', 'odang_person', 'triples_verbalised', 'name', 'from_LM'])
#concateno text e triples_verbalised
#davinci['text'] = davinci['text'].str.cat(davinci['triples_verbalised'], sep=' ')
#davinci = davinci.drop(columns=['Unnamed: 0', 'o', 'odang_person', 'triples_verbalised', 'name', 'from_LM'])
train = pd.read_excel('haspeedee_train.xlsx')
train.rename(columns={'anonymized_tweet_id':'s'}, inplace=True)
train.rename(columns={'anonymized_text':'text'}, inplace=True)
train.drop(columns=['dataset', 'data_type'], inplace=True)

def replace_text(davinci, train):
    # Merge the 'train' and 'davinci' datasets based on 'ID'
    merged_dataset = pd.merge(train, davinci[['s', 'text']], on='s', how='left')

    # Update the 'text' column of 'train' with the values from 'davinci' where IDs match
    merged_dataset['text'] = merged_dataset['text_y'].combine_first(merged_dataset['text_x'])

    # Drop the extra columns
    merged_dataset = merged_dataset.drop(['text_x', 'text_y'], axis=1)

    return merged_dataset

    
merged_data = replace_text(davinci, train)
merged_data.drop_duplicates(subset='s', keep='first', inplace=True)
data = merged_data

#Training misogyny

In [ ]:
#baseline
data = pd.read_excel('haspeedee_train.xlsx')
data.rename(columns={'anonymized_tweet_id':'s'}, inplace=True)
data.rename(columns={'anonymized_text':'text'}, inplace=True)
data.drop(columns=['dataset', 'data_type'], inplace=True)

In [ ]:
#aggiungo definizione
data['text'] = data['text'].apply(lambda x: x + 'Per discorso d\'odio si intende qualsiasi tipo di comunicazione che attacchi o utilizzi un linguaggio peggiorativo o discriminatorio nei confronti di una persona o di un gruppo in base alla sua identità, ovvero in base alla sua religione, etnia, nazionalità, razza, colore, discendenza, sesso o altro fattore di identità.')

In [ ]:
data = unique

In [ ]:
data

,s,label,text
0,217874450618134,1,@unknown Con tutte le denunce che si sta becca...
1,740684329660693,0,In pochi giorni il #ministro #salvini ha butta...
2,170378636015491,1,Ma se la #carfagna e #crosetto sono più a #sin...
3,906842219275344,1,Nemmeno la tradizionale conferenza stampa di f...
5,48581434415291,1,#decretosicurezza #LeolucaOrlando colui che er...
...,...,...,...
553,79052048967937,1,"@943206476637778 dice in #Senato che ""la #leg..."
554,536376311847245,1,La lega raccoglie 90000 firme per togliere il ...
555,324165730844406,1,".@738958761853559 , ma vafangul’\n\nmessaggio ..."
556,871453511657474,1,#ENRICOLETTA É DIARREA BERLUSCONICCHIA CHE POR...


In [25]:
from sklearn.model_selection import train_test_split

# Split dataset in traning and validation(test)
X_train, X_val, Y_train, Y_val = train_test_split(
    data.index.values,
    data.label.values,
    test_size=0.10,
    random_state=17,
    stratify=data.label.values
)

In [26]:
# Check datasets composition
data['data_type'] = ['not_set'] * data.shape[0]
data.loc[X_train, 'data_type'] = 'train'
data.loc[X_val, 'data_type'] = 'val'
data.groupby(['label', 'data_type']).count()

s  text
label data_type            
0     train      2650  2650
      val         806   806
1     train      4205  4205
      val        1104  1104

In [27]:
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import TensorDataset


tokenizer = AutoTokenizer.from_pretrained("m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0", use_fast=False)

In [28]:
# Encode training dataset using the tokenizer
encoded_data_train = tokenizer.batch_encode_plus(
    data[data.data_type == 'train'].text.values,
    add_special_tokens=True,
    return_attention_mask=True, 
    pad_to_max_length=True,
    max_length=512,
    return_tensors='pt'
)

# Encode validation dataset using the tokenizer
encoded_data_val = tokenizer.batch_encode_plus(
    data[data.data_type == 'val'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,  
    pad_to_max_length=True,
    max_length=512,
    return_tensors='pt'
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [29]:
# Extract IDs, attention masks and labels from training dataset
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(data[data.data_type == 'train'].label.values)
labels_train

tensor([1, 1, 1,  ..., 1, 1, 1])

In [30]:
# Extract IDs, attention masks and labels from validation dataset
input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(data[data.data_type == 'val'].label.values)

In [31]:
# Create train and validation dataset from extracted features
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)
print("Train dataset length: {}\nValidation dataset length: {}".format(len(dataset_train), len(dataset_val)))

Train dataset length: 6855
Validation dataset length: 1910


In [32]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# Define the size of each batch
batch_size = 16

# Load training dataset
dataloader_train= DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size)

# Load valuation dataset
dataloader_val= DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=batch_size)

In [33]:
from transformers import BertForSequenceClassification
# Load pre-trained BERT model
model = BertForSequenceClassification.from_pretrained("m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0")

Some weights of the model checkpoint at m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0 were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were

In [34]:
from transformers import AdamW, get_linear_schedule_with_warmup

# Define model optimizer -> Adam
optimizer = AdamW(
    model.parameters(),
    lr = 1e-5, 
    eps=1e-8
)
# Define model scheduler
epochs = 4
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [35]:
import random

# Define random seeds
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [36]:
# Define processor type for torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
device

device(type='cuda')

In [37]:
from sklearn.metrics import f1_score, precision_score, recall_score

# Returns the F1 score computed on the predictions
def f1_score_func(preds, labels):
    preds_flat=np.argmax(preds, axis=1).flatten()
    labels_flat=labels.flatten()
    return f1_score(labels_flat, preds_flat, average='macro')



# Returns the precision and recall score computed on the predictions
def prec_func(preds, labels):
    preds_flat=np.argmax(preds, axis=1).flatten()
    labels_flat=labels.flatten()
    return precision_score(labels_flat, preds_flat, average='macro')

def recall_func(preds, labels):
    preds_flat=np.argmax(preds, axis=1).flatten()
    labels_flat=labels.flatten()
    return recall_score(labels_flat, preds_flat, average='macro')



In [38]:
# Evaluates the model using the validation set
def evaluate(dataloader_val):
  model.eval()
  loss_val_total = 0
  predictions, true_vals = [], []

  for batch in dataloader_val:
      batch = tuple(b.to(device) for b in batch)
      inputs = {'input_ids': batch[0],
        'attention_mask': batch[1],
        'labels': batch[2],
        }

      with torch.no_grad():
          outputs = model(**inputs)

      loss = outputs[0]
      logits = outputs[1]
      loss_val_total += loss.item()

      logits = logits.detach().cpu().numpy()
      label_ids = inputs['labels'].cpu().numpy()
      predictions.append(logits)
      true_vals.append(label_ids)

  loss_val_avg = loss_val_total / len(dataloader_val)

  predictions = np.concatenate(predictions, axis=0)
  true_vals = np.concatenate(true_vals, axis=0)

  return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs + 1)):

    model.train()  # model is training

    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'labels': batch[2]}
        
        outputs = model(**inputs)

        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()  # to backpropagate

        torch.nn.utils.clip_grad_norm_(model.parameters(),
                                      1.0)  # prevents the gradient from being too small or too big

        optimizer.step()
        scheduler.step()
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item() / len(batch))})

    
    torch.save(model, path)
    tqdm.write(f'\nEpoch {epoch}/{epochs}')

    loss_train_avg = loss_train_total / len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')  # make sure that model is still training

    val_loss, predictions, true_vals = evaluate(dataloader_val)  # to check overtraining (or overfitting)
    val_f1 = f1_score_func(predictions, true_vals)
    val_prec = prec_func(predictions, true_vals)
    val_recall = recall_func(predictions, true_vals)

    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score(weighted) : {val_f1}')
    tqdm.write(f'Prec Score(weighted) : {val_prec}')
    tqdm.write(f'Recall Score(weighted) : {val_recall}')


  0%|          | 0/4 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/429 [00:00<?, ?it/s]

In [ ]:
model.save_pretrained('alberto_haspeede_verbalizz_conan_testsetuff')

#Preds sul test

In [ ]:
#Load test set
data_test = pd.read_csv('haspeede_test_subtaskB.csv')
data_test

In [ ]:
data_test = data_test.rename(columns={'anonymized_tweet_id':'s', 'anonymized_text':'text'})
data_test = data_test.drop(columns=['dataset'])

In [ ]:
merged_datatest = replace_text(davinci, data_test)
merged_datatest.drop_duplicates(subset='s', keep='first', inplace=True)
data_test = merged_datatest

In [ ]:
#definizione
data_test['text'] = data_test['text'].apply(lambda x: x + 'Per discorso d\'odio si intende qualsiasi tipo di comunicazione che attacchi o utilizzi un linguaggio peggiorativo o discriminatorio nei confronti di una persona o di un gruppo in base alla sua identità, ovvero in base alla sua religione, etnia, nazionalità, razza, colore, discendenza, sesso o altro fattore di identità.')

In [ ]:
# Encode validation dataset using the tokenizer
encoded_data_test = tokenizer.batch_encode_plus(
    data_test.text.values,
    add_special_tokens=True,
    return_attention_mask=True,  
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)


In [ ]:
# Extract IDs, attention masks and labels from validation dataset
input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']

dataset_test = TensorDataset(input_ids_test, attention_masks_test)
print("Test dataset length: {}".format(len(dataset_test)))

In [ ]:
from torch.utils.data import DataLoader
dataloader_test = DataLoader(dataset_test)

In [ ]:
# Evaluates the model using the validation set
def predict(dataset_test):
    predictions = []

    for row in dataset_test:
      row = tuple(r.to(device) for r in row)
      inputs = {'input_ids': row[0],
        'attention_mask': row[1]
        }

      with torch.no_grad():
          outputs = model(**inputs)

      logits = outputs[0]
      logits = logits.detach().cpu().numpy()
      predictions.append(logits)

    return predictions

# Predict values for test dataset
predictions = predict(dataloader_test)

In [ ]:
print(len(predictions))
results = []
for i, prediction in enumerate(predictions):
  predicted = np.argmax(prediction, axis=1)[0]
  # print(f"index: {i} -- prediction: {predicted}")
  results.append(predicted)

print(results)
print(results.count(0))
print(results.count(1))

In [ ]:
misogynous = []
for prediction in results:
  misogynous.append(prediction)

In [ ]:
#Predict misogyny
data_test['preds'] = misogynous


In [ ]:
#Save the predicted labels in a csv file 
data_test.to_excel('preds_haspeedee_taskB_verb+conan.xlsx', index=False)

In [ ]:
from google.colab import files
files.download('preds_haspeedee_taskB_verb+conan.xlsx')

#Testing 

In [ ]:
import pandas as pd
import sklearn
from sklearn.metrics import classification_report

In [ ]:
data_test = pd.read_excel('preds_haspeedee_dev_davinci_def.xlsx')

In [ ]:
#hs = pd.read_excel('preds_haspeedee_test_davinci.xlsx')

In [ ]:
data_test

,s,text,label,preds
0,172580609652325,La #Lega e il #m5s stanno dando a bere all'opi...,0,0
1,825279779951638,Il decreto #salvini crea insicurezza e disuman...,0,0
2,863998520896761,"""Paolo Polidori"" dovrebbe dimettersi per manif...",1,0
3,327244336972419,#ottoemezzo. #Bongiorno ci racconta che il gov...,0,0
4,897329723892923,#ottoemezzo #cacciari spiega alla #bongiorno l...,0,0
...,...,...,...,...
555,324165730844406,".@738958761853559 , ma vafangul’\n\nmessaggio ...",1,1
556,871453511657474,#ENRICOLETTA É DIARREA BERLUSCONICCHIA CHE POR...,1,1
557,67223505911646,"Ultima Ora: Webuild, nuovi ordini per 2,6 mili...",0,0
558,517002584005250,#14maggio #PapaFrancesco Occorrono politiche f...,0,0


In [ ]:
y_true = data_test['label'].to_numpy()
y_preds = data_test['preds'].to_numpy()
print(sklearn.metrics.classification_report(y_true, y_preds))

              precision    recall  f1-score   support

           0       0.95      0.84      0.89       346
           1       0.79      0.93      0.85       214

    accuracy                           0.88       560
   macro avg       0.87      0.89      0.87       560
weighted avg       0.89      0.88      0.88       560



#Da Vinci

In [ ]:
data = pd.read_excel('haspeedee_train.xlsx')
kg = pd.read_excel('davinci002_knowledge.xlsx')
kg

,odang_person,name,from_LM
0,odang_person_7200101,Matteo Salvini,"Il leader della Lega Nord, Matteo Salvini, è p..."
1,odang_person_3585484,Matteo Renzi,"Nel corso degli ultimi anni, Matteo Renzi è st..."
2,odang_person_7200508,Paolo Gentiloni,Non è possibile determinare la probabilità che...
3,odang_person_3592345,Paolo Conte,Non è possibile determinare con certezza la pr...
4,odang_person_276429,Luigi Di Maio,Non è possibile quantificare la probabilità ch...
...,...,...,...
686,odang_person_4797813,Francesco Acerbi,Non è possibile determinare la probabilità che...
687,odang_person_77927,Gianluca Signorini,Non è possibile determinare con precisione la ...
688,odang_person_2318873,Tommaso Zorzi,Non è possibile stabilire con certezza la prob...
689,odang_person_1300109,Claudia Pandolfi,Mentre non è possibile prevedere con certezza ...


In [ ]:
verbalised = pd.read_excel('kg4haspeede_verbalised.xlsx')
verbalised.rename(columns={'entity':'odang_person'}, inplace=True)

In [ ]:
verbalised

,s,o,odang_person,triples_verbalised
0,217874450618134,refersTo,odang_person_7200101,Matteo Salvini nacque a Milano. nacque nel 1...
1,514513704562272,refersTo,odang_person_7200101,Matteo Salvini nacque a Milano. nacque nel 1...
2,187067185890488,refersTo,odang_person_7200101,Matteo Salvini nacque a Milano. nacque nel 1...
3,775307119996545,refersTo,odang_person_7200101,Matteo Salvini nacque a Milano. nacque nel 1...
4,552761280040245,refersTo,odang_person_7200101,Matteo Salvini nacque a Milano. nacque nel 1...
...,...,...,...,...
2374,767572489901726,refersTo,odang_person_4881081,Elisabetta Belloni parla e scrive italiano. n...
2375,62598046102406,refersTo,odang_person_4801710,Giuliano Pisapia nacque a Milano. è maschio....
2376,947809243804071,refersTo,odang_person_8766096,Pasquale Martignetti nacque a Benevento. morì...
2377,531900858879345,refersTo,odang_person_3594357,Diego Simeone è genitore di Giovanni Simeone....


In [ ]:
merged = pd.merge(verbalised, kg, on=['odang_person', 'odang_person'])

# save the merged dataset to a new file
merged.to_excel('common_dataset.xlsx', index=False)

In [ ]:
merged

,s,o,odang_person,triples_verbalised,name,from_LM
0,217874450618134,refersTo,odang_person_7200101,Matteo Salvini nacque a Milano. nacque nel 1...,Matteo Salvini,"Il leader della Lega Nord, Matteo Salvini, è p..."
1,514513704562272,refersTo,odang_person_7200101,Matteo Salvini nacque a Milano. nacque nel 1...,Matteo Salvini,"Il leader della Lega Nord, Matteo Salvini, è p..."
2,187067185890488,refersTo,odang_person_7200101,Matteo Salvini nacque a Milano. nacque nel 1...,Matteo Salvini,"Il leader della Lega Nord, Matteo Salvini, è p..."
3,775307119996545,refersTo,odang_person_7200101,Matteo Salvini nacque a Milano. nacque nel 1...,Matteo Salvini,"Il leader della Lega Nord, Matteo Salvini, è p..."
4,552761280040245,refersTo,odang_person_7200101,Matteo Salvini nacque a Milano. nacque nel 1...,Matteo Salvini,"Il leader della Lega Nord, Matteo Salvini, è p..."
...,...,...,...,...,...,...
2373,767572489901726,refersTo,odang_person_4881081,Elisabetta Belloni parla e scrive italiano. n...,Elisabetta Belloni,Non è possibile quantificare la probabilità ch...
2374,62598046102406,refersTo,odang_person_4801710,Giuliano Pisapia nacque a Milano. è maschio....,Giuliano Pisapia,Giuliano Pisapia è vittima di un commento onli...
2375,947809243804071,refersTo,odang_person_8766096,Pasquale Martignetti nacque a Benevento. morì...,Pasquale Martignetti,It is probable that Pasquale Martignetti will ...
2376,531900858879345,refersTo,odang_person_3594357,Diego Simeone è genitore di Giovanni Simeone....,Diego Simeone,Diego Simeone è probabile vittima di un commen...


In [ ]:
data

,anonymized_tweet_id,anonymized_text,label,dataset,data_type
0,217874450618134,@unknown Con tutte le denunce che si sta becca...,1,training_politics,train
1,360042217507605,#PRESCRIZIONE : I (t)re magi hanno trovato l’a...,0,training_politics,train
2,817917484817935,Il #m5s ha votato contro le #unionicivili addu...,1,training_politics,train
3,145836038456701,Che cosa cambia questa #legge? Caro @667517147...,0,training_politics,train
4,599642342348521,@800991101440753 Spero rientrino 2000 insegnan...,0,training_politics,train
...,...,...,...,...,...
5035,221538539806447,Sportello telematico attivo al Comune di Santa...,0,training_politics,train
5036,715670659156960,#Covid. #Draghi non fare come #Conte. La #poli...,0,training_politics,train
5037,512309888296620,"Cina, «128mila euro a neonato»: la proposta pe...",0,training_politics,train
5038,887061502405620,@401560935985106 @507718538254873 @28429789767...,1,training_politics,train


In [ ]:
#join text and triples
merged['text'] = data['anonymized_text'].str.cat(merged['from_LM'], sep=' ')
merged.to_excel('haspeede_verbalis_davinci.xlsx')

In [ ]:
merged

,s,o,odang_person,triples_verbalised,name,from_LM,text
0,217874450618134,refersTo,odang_person_7200101,Matteo Salvini nacque a Milano. nacque nel 1...,Matteo Salvini,"Il leader della Lega Nord, Matteo Salvini, è p...",@unknown Con tutte le denunce che si sta becca...
1,514513704562272,refersTo,odang_person_7200101,Matteo Salvini nacque a Milano. nacque nel 1...,Matteo Salvini,"Il leader della Lega Nord, Matteo Salvini, è p...",#PRESCRIZIONE : I (t)re magi hanno trovato l’a...
2,187067185890488,refersTo,odang_person_7200101,Matteo Salvini nacque a Milano. nacque nel 1...,Matteo Salvini,"Il leader della Lega Nord, Matteo Salvini, è p...",Il #m5s ha votato contro le #unionicivili addu...
3,775307119996545,refersTo,odang_person_7200101,Matteo Salvini nacque a Milano. nacque nel 1...,Matteo Salvini,"Il leader della Lega Nord, Matteo Salvini, è p...",Che cosa cambia questa #legge? Caro @667517147...
4,552761280040245,refersTo,odang_person_7200101,Matteo Salvini nacque a Milano. nacque nel 1...,Matteo Salvini,"Il leader della Lega Nord, Matteo Salvini, è p...",@800991101440753 Spero rientrino 2000 insegnan...
...,...,...,...,...,...,...,...
2373,767572489901726,refersTo,odang_person_4881081,Elisabetta Belloni parla e scrive italiano. n...,Elisabetta Belloni,Non è possibile quantificare la probabilità ch...,La Camera riavvia legge cannabis. Lega vuole s...
2374,62598046102406,refersTo,odang_person_4801710,Giuliano Pisapia nacque a Milano. è maschio....,Giuliano Pisapia,Giuliano Pisapia è vittima di un commento onli...,#ioapro.Forza ragazzi!Disobbedite ai #dpcm.Pe...
2375,947809243804071,refersTo,odang_person_8766096,Pasquale Martignetti nacque a Benevento. morì...,Pasquale Martignetti,It is probable that Pasquale Martignetti will ...,"Secondo un retroscena de #IlMessaggero, il pre..."
2376,531900858879345,refersTo,odang_person_3594357,Diego Simeone è genitore di Giovanni Simeone....,Diego Simeone,Diego Simeone è probabile vittima di un commen...,@465909497202033 @323218462168424 @48825281464...


#Error Analysis

In [ ]:
#Error analysis
error = hs[hs['label'] != hs['preds']]
error.label.value_counts() #num di FP e FN è bilanciato, + 6 FP

0    27
1    21
Name: label, dtype: int64

In [ ]:
import nltk
text_list = error['anonymized_text'].tolist()
#remove stopwords
nltk.download('stopwords')
nltk.download('punkt')

stopwords = set(nltk.corpus.stopwords.words('italian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def convert(lst):
    return ' '.join(lst).split()
     
print(convert(text_list))

['"Paolo', 'Polidori"', 'dovrebbe', 'dimettersi', 'per', 'manifesta', 'incapacità:', 'invece', 'di', 'provvedere', 'nei', 'modi', 'consentiti', 'dalla', '#legge', 'e', 'dalla', 'sua', 'funzione', 'di', '#vicesindaco', 'risolvendo', 'la', 'situazione', 'di', 'disagio,', 'ha', 'preferito', 'agire', 'da', 'vandalo.', 'Non', 'merita', 'di', 'stare', 'in', 'un', 'Consiglio', 'Comunale.', '#Trieste', '@868843245148057', 'W', 'il', '#m5s', 'No', 'alla', '#casta', 'dei', 'partiti.', 'No', 'al', '#malaffare', '@861285225934020', '#salvinidimettiti', 'e', '#salvinifattiprocessare', 'perché', 'siamo', 'tutti', 'uguali', 'davanti', 'alla', 'legge', 'e', 'la', '#Magistratura', 'è', 'ancora', 'indipendente.', 'Perché', '#siamoumani', 'e', 'basta', '#Diciotti', '#facciamorete', 'Se', 'la', '#SeaWatch', 'fosse', 'piena', 'di', 'gattini', 'sareste', 'già', 'partiti', 'con', 'i', 'pedalò', 'per', 'salvarli.', 'Mi', 'disgustate.', '#facciamorete', '#PortiAperti', '#salvinidimettiti', '#restiamoumani', 'I

In [ ]:
split_text = convert(text_list)
allWordExceptStopDist = nltk.FreqDist(w.lower() for w in split_text if w not in stopwords) 

In [ ]:
mostCommon= allWordExceptStopDist.most_common(120)
mostCommon

[('#m5s', 7),
 ('la', 6),
 ('senza', 6),
 ('[url]', 6),
 ('dovrebbe', 4),
 ('invece', 4),
 ('#legge', 4),
 ('non', 4),
 ('no', 4),
 ('perché', 4),
 ('#conte', 4),
 ('sono', 4),
 (',', 4),
 ('in', 4),
 ('a', 4),
 ('#renzi', 4),
 ('#draghi', 4),
 ('#salvinidimettiti', 3),
 ('legge', 3),
 ('#facciamorete', 3),
 ('governo', 3),
 ('fine', 3),
 ('fare', 3),
 ('#governo', 3),
 ('#salvini', 3),
 ('@943206476637778', 3),
 ('#salvinivergognati', 3),
 ('mai', 3),
 ('.', 3),
 ('avanti', 3),
 ('essere', 3),
 ('#pd', 3),
 ('che', 3),
 ('salvini', 3),
 ('può', 3),
 ('solo', 3),
 ('@804831429960458', 3),
 ('via', 3),
 ('sempre', 3),
 ('e', 3),
 ('#speranzadimettiti', 3),
 ('#letta', 3),
 ('stare', 2),
 ('w', 2),
 ('ancora', 2),
 ('#diciotti', 2),
 ('se', 2),
 ('mi', 2),
 ('#portiaperti', 2),
 ('va', 2),
 ('mano', 2),
 ('eh', 2),
 ('niente', 2),
 ('c’è', 2),
 ('#ministro', 2),
 ('bene', 2),
 ('diritto', 2),
 ("c'è", 2),
 ('quindi', 2),
 ('problema', 2),
 ('#roma', 2),
 ('ma', 2),
 ('strada', 2),
 ('gli

In [ ]:
error

,anonymized_tweet_id,anonymized_text,label,dataset,data_type,preds
2,863998520896761,"""Paolo Polidori"" dovrebbe dimettersi per manif...",1,training_politics,val,0
5,848965724551157,@868843245148057 W il #m5s\n\nNo alla #casta d...,0,training_politics,val,1
7,283508205494975,@861285225934020 #salvinidimettiti e #salvinif...,0,training_politics,val,1
8,933799779508782,Se la #SeaWatch fosse piena di gattini sareste...,0,training_politics,val,1
10,229647197643357,Intanto in #Spagna il governo socialista di #S...,1,training_politics,val,0
11,733879466972060,Sono iscritto al #M5S dal 2010 e voterò NO per...,1,training_politics,val,0
23,71184870630357,#Rami eh niente c’è un #Ministro che non sta b...,0,training_politics,val,1
27,820433310798508,Se credete che la #legge sul #RevengePorn sia ...,0,training_politics,val,1
56,494786086468365,Quelli che si lamentano della puzza di urina p...,1,training_politics,val,0
63,624468506805769,@943206476637778 I processi si fanno in tribun...,0,training_politics,val,1


Patterns di errori: 

FALSE POSITIVES:

1) Frasi contenenti sentiment negativo ma verso statements razzisti (Se la #SeaWatch fosse piena di gattini sareste già partiti con i pedalò per salvarli. *Mi disgustate.* #facciamorete #PortiAperti #salvinidimettiti #restiamoumani)

FALSE NEGATIVES:

1) Soggetto implicito (Quelli che si lamentano della puzza di urina per le strade di #Roma sono gli stessi che dicono #Fateliscendere o #portiaperti ! Ma secondo loro chi cazzo è che *piscia per strada a tutte le ore, che vagano ubriachi o senza meta?* Gli alieni? #raggi #salvini #facciamorete)


